In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

# assign to first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#VRAM memory on-demand usage(prevent Keras takes all memory at the beginning)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 40
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_main, y_main), (x_test, y_test) = cifar10.load_data()

print('x shape:', x_main.shape)
x_train=x_main[:40000]
y_train=y_main[:40000]
print('x_train shape:', x_train.shape)
x_val=x_main[40000:]
y_val=y_main[40000:]
print('x_val shape:', x_val.shape)

x shape: (50000, 32, 32, 3)
x_train shape: (40000, 32, 32, 3)
x_val shape: (10000, 32, 32, 3)


In [3]:


# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [5]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255

In [6]:


print('Not using data augmentation.')
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_val, y_val),
            shuffle=True)


Not using data augmentation.
Train on 40000 samples, validate on 10000 samples
Epoch 1/40
40000/40000 [==============================] - 13s 329us/step - loss: 1.6921 - acc: 0.3912 - val_loss: 1.4875 - val_acc: 0.4563
Epoch 2/40
40000/40000 [==============================] - 11s 265us/step - loss: 1.3910 - acc: 0.5044 - val_loss: 1.3054 - val_acc: 0.5373
Epoch 3/40
40000/40000 [==============================] - 11s 264us/step - loss: 1.2449 - acc: 0.5581 - val_loss: 1.2290 - val_acc: 0.5623
Epoch 4/40
40000/40000 [==============================] - 10s 262us/step - loss: 1.1321 - acc: 0.6016 - val_loss: 1.1560 - val_acc: 0.5955
Epoch 5/40
40000/40000 [==============================] - 11s 263us/step - loss: 1.0411 - acc: 0.6342 - val_loss: 1.0493 - val_acc: 0.6385
Epoch 6/40
40000/40000 [==============================] - 11s 263us/step - loss: 0.9680 - acc: 0.6634 - val_loss: 0.9806 - val_acc: 0.6609
Epoch 7/40
40000/40000 [==============================] - 11s 267us/step - loss: 0.9053

In [7]:
x_test = x_test.astype('float32')
x_test /= 255
y_test = keras.utils.to_categorical(y_test, num_classes)
model.evaluate(x=x_test, y=y_test, batch_size=batch_size, verbose=1)

10000/10000 [==============================] - 1s 82us/step


[2.0158658267974854, 0.7219]

In [8]:
model.save('simple.h5')